In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from pandasql import sqldf
from sklearn.metrics import mean_squared_error

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_excel('C:\\Users\\osungar\\Desktop\projects\\sales_forecasting\\sales_project\\data\\satis_new.xlsx')


In [4]:
df.head()

,CARI_NO,CARI_ADI,STOK_NO,STOK_KODU,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI,DVZ_CINSI,IMDSC1,IMSRP1SDC,IMSRP2SDC,IMSRP3SDC,IMSRP4SDC,IMSRP5SDC,IMSRP6SDC,IMSRP7SDC,IMSRP8SDC,IMSRP9SDC,IMSRP0SDC,IMPRP1DSC,IMPRP2DSC,IMPRP3DSC,IMPRP4DSC,IMPRP5DSC,IMPRP6DSC,IMPRP7DSC,IMPRP8DSC,IMPRP9DSC,IMPRP0DSC,IMSHCNDSC,IMSHCMDSC
0,10636,MEKLAS OTOMOTİV İTHALAT İHRACAT SANAYİ,1743,MML.TKN.KAC.MEL.019,303.75,KG,7.000,AD,2015-04-01,USD,"Mek 575mm 0,9mm-NYL-411012-11",TEKNİK HAMUR,0,0,0,0,0,MEKLAS,ŞEKİLLENDİRME,0,0,Mamul,0,0,Ana Urun grubu,0,Karışım Siyah,Parafinik Yağlı,KORUK,0,ASTAR,KALENDER CIKISLI,.
1,10657,VIBRACOUSTIC ÇERKEZKÖY OTOMOTİV SAN.TİC.,743090,MML.TKN.BAC.TRB.124,198.00,KG,1.564,B3,2015-04-01,EUR,Vibracoustic WAR 4500,TEKNİK HAMUR,0,0,0,0,Otomatik İş Emri,TRELLEBORG TBVC BURSA,KARISIM,0,Iki Kademe,Mamul,0,0,Ana Urun grubu,0,Karışım Renkli,Parafinik Yağlı,ANTIVIBRASYON,0,0,BANBURY CIKISLI,.
2,10636,MEKLAS OTOMOTİV İTHALAT İHRACAT SANAYİ,1737,MML.TKN.KAC.MEL.013,1558.50,KG,30.000,AD,2015-04-01,USD,"Mek 665mm 1,2mm-NYL-411042-11",TEKNİK HAMUR,0,0,0,0,0,MEKLAS,ŞEKİLLENDİRME,0,0,Mamul,0,0,Ana Urun grubu,0,Karışım Siyah,Parafinik Yağlı,KORUK,0,ASTAR,KALENDER CIKISLI,.
3,10657,VIBRACOUSTIC ÇERKEZKÖY OTOMOTİV SAN.TİC.,1979,MML.TKN.BAC.TRB.018,268.00,KG,1.000,B3,2015-04-01,EUR,Vibracoustic T 057,TEKNİK HAMUR,0,0,0,0,Otomatik İş Emri,TRELLEBORG TBVC BURSA,KARISIM,0,Üç Kademe,Mamul,0,0,Ana Urun grubu,0,Karışım Renkli,Parafinik + Antilüks 111,ANTIVIBRASYON,0,0,BANBURY CIKISLI,.
4,10636,MEKLAS OTOMOTİV İTHALAT İHRACAT SANAYİ,1736,MML.TKN.KAC.MEL.012,762.00,KG,23.000,AD,2015-04-01,USD,"Mek 370mm 1,4mm-NYL-411028-11",TEKNİK HAMUR,0,0,0,0,0,MEKLAS,ŞEKİLLENDİRME,0,0,Mamul,0,0,Ana Urun grubu,0,Karışım Siyah,Parafinik Yağlı,KORUK,0,ASTAR,KALENDER CIKISLI,.


In [5]:
query = """
SELECT STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI                 
FROM df
"""

# Run the query
df_dropped = sqldf(query, locals())
df_dropped.head()

,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI
0,1743,303.75,KG,7.000,AD,2015-04-01 00:00:00.000000
1,743090,198.00,KG,1.564,B3,2015-04-01 00:00:00.000000
2,1737,1558.50,KG,30.000,AD,2015-04-01 00:00:00.000000
3,1979,268.00,KG,1.000,B3,2015-04-01 00:00:00.000000
4,1736,762.00,KG,23.000,AD,2015-04-01 00:00:00.000000


In [6]:
df_dropped.tail()

,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI
243846,756313,7.00,AD,102.15,KG,2023-10-30 00:00:00.000000
243847,2897,51.25,KG,1.00,AD,2023-10-30 00:00:00.000000
243848,2966,249.85,KG,5.00,AD,2023-10-30 00:00:00.000000
243849,2980,115.95,KG,5.00,AD,2023-10-30 00:00:00.000000
243850,2913,48.60,KG,1.00,AD,2023-10-30 00:00:00.000000


In [7]:
df_dropped["SATIS_TARIHI"] = df_dropped["SATIS_TARIHI"].astype("datetime64[ns]")


In [8]:
df_dropped.tail()

,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI
243846,756313,7.00,AD,102.15,KG,2023-10-30
243847,2897,51.25,KG,1.00,AD,2023-10-30
243848,2966,249.85,KG,5.00,AD,2023-10-30
243849,2980,115.95,KG,5.00,AD,2023-10-30
243850,2913,48.60,KG,1.00,AD,2023-10-30


In [9]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243851 entries, 0 to 243850
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   STOK_NO            243851 non-null  int64         
 1   TOPLAM_BIRINCI_OB  243851 non-null  float64       
 2   OB1                243851 non-null  object        
 3   TOPLAM_IKINCI_OB   243851 non-null  float64       
 4   OB2                243851 non-null  object        
 5   SATIS_TARIHI       243851 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 11.2+ MB


In [10]:
df_date = df_dropped.copy()

In [11]:
df_date['YIL'] = df_date['SATIS_TARIHI'].dt.year
df_date['AY'] = df_date['SATIS_TARIHI'].dt.month

In [12]:
df_date = df_date.drop(columns=['SATIS_TARIHI'])

In [13]:
df_date.head()

,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,YIL,AY
0,1743,303.75,KG,7.000,AD,2015,4
1,743090,198.00,KG,1.564,B3,2015,4
2,1737,1558.50,KG,30.000,AD,2015,4
3,1979,268.00,KG,1.000,B3,2015,4
4,1736,762.00,KG,23.000,AD,2015,4


In [14]:
df_date_kg = df_date[df_date['OB1']=='KG']

In [15]:
df_date_kg['OB1'].nunique()

1

In [16]:
df_date_kg = df_date_kg.drop(columns=['OB1'])

In [17]:
df_date_kg.tail()

,STOK_NO,TOPLAM_BIRINCI_OB,TOPLAM_IKINCI_OB,OB2,YIL,AY
243843,754392,2062.80,40.0,AD,2023,10
243847,2897,51.25,1.0,AD,2023,10
243848,2966,249.85,5.0,AD,2023,10
243849,2980,115.95,5.0,AD,2023,10
243850,2913,48.60,1.0,AD,2023,10


In [18]:
df_date_kg_ad = df_date_kg[df_date_kg['OB2']=='AD']

In [19]:
df_date_kg_ad = df_date_kg_ad.drop(columns=['OB2'])

In [20]:
df_date_kg_ad.tail()

,STOK_NO,TOPLAM_BIRINCI_OB,TOPLAM_IKINCI_OB,YIL,AY
243843,754392,2062.80,40.0,2023,10
243847,2897,51.25,1.0,2023,10
243848,2966,249.85,5.0,2023,10
243849,2980,115.95,5.0,2023,10
243850,2913,48.60,1.0,2023,10


# Aşağıdaki kod ile OB2'yi ekleyip çıkar.
SELECT STOK_NO,AY,YIL,AVG(TOPLAM_IKINCI_OB) as OB2_AYLIK_ORTALAMA,AVG(TOPLAM_BIRINCI_OB) as OB1_AYLIK_ORTALAMA        


In [22]:
query = """
SELECT STOK_NO,AY,YIL,AVG(TOPLAM_BIRINCI_OB) as OB1_AYLIK_ORTALAMA        
FROM df_date_kg_ad
group by STOK_NO,YIL,AY
order by YIL,AY ASC
"""

# Run the query
df_month_kg_ad = sqldf(query, locals())
df_month_kg_ad.head(10)

,STOK_NO,AY,YIL,OB1_AYLIK_ORTALAMA
0,1385,4,2015,75.350000
1,1387,4,2015,25.400000
2,1388,4,2015,24.500000
3,1391,4,2015,418.800000
4,1392,4,2015,46.866667
5,1394,4,2015,21.500000
6,1407,4,2015,344.600000
7,1408,4,2015,229.800000
8,1410,4,2015,287.500000
9,1412,4,2015,628.600000


# rolling


## rolling raw code

In [75]:
import pandas as pd

# Örnek bir DataFrame oluştur
data = {'STOK_NO': [1, 1, 1, 2, 2, 2, 3, 3, 3],
        'AY': [3, 1, 2, 2, 1, 6, 2, 1, 9],
        'YIL': [2001, 2022, 1980, 2022, 2022, 2021, 2022, 2023, 2022],
        'OB1_AYLIK_ORTALAMA': [10, 15, 20, 5, 10, 15, 8, 12, 16]}

df = pd.DataFrame(data)

# STOK_NO, YIL ve AY sütunlarına göre sıralama
df = df.sort_values(by=['STOK_NO', 'YIL', 'AY'])

# STOK_NO'ya göre gruplama
grouped = df.groupby('STOK_NO')

# Her bir grup üzerinde 3 aylık pencere üzerinde rolling işlemi yapma
df['3_Aylık_Rolling'] = grouped['OB1_AYLIK_ORTALAMA'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True).shift(1).fillna(0)

# Sonuçları göster
df


,STOK_NO,AY,YIL,OB1_AYLIK_ORTALAMA,3_Aylık_Rolling
2,1,2,1980,20,0.0
0,1,3,2001,10,20.0
1,1,1,2022,15,15.0
5,2,6,2021,15,15.0
4,2,1,2022,10,15.0
3,2,2,2022,5,12.5
6,3,2,2022,8,10.0
8,3,9,2022,16,8.0
7,3,1,2023,12,12.0


In [61]:
import pandas as pd

# Örnek DataFrame oluşturulması (gerçek veri çerçevenizi kullanın)
data = {'STOK_NO': [1, 1, 1, 2, 2, 2, 3, 3, 3],
        'AY': [3, 1, 2, 2, 1, 6, 2, 1, 9],
        'YIL': [2001, 2022, 1980, 2022, 2022, 2021, 2022, 2023, 2022],
        'OB1_AYLIK_ORTALAMA': [10, 15, 20, 5, 10, 15, 8, 12, 16]}

df = pd.DataFrame(data)

# STOK_NO, YIL ve AY sütunlarına göre sıralama
df = df.sort_values(by=['STOK_NO', 'YIL', 'AY'])

# STOK_NO'ya göre gruplama
grouped = df.groupby('STOK_NO')



# Her bir grup üzerinde 3 aylık pencere üzerinde rolling işlemi yapma
df['3_Aylık_Rolling'] = grouped['OB1_AYLIK_ORTALAMA'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True).shift(1)




# Sonuçları gösterme
df


,STOK_NO,AY,YIL,OB1_AYLIK_ORTALAMA,3_Aylık_Rolling
2,1,2,1980,20,NaN
0,1,3,2001,10,20.0
1,1,1,2022,15,15.0
5,2,6,2021,15,15.0
4,2,1,2022,10,15.0
3,2,2,2022,5,12.5
6,3,2,2022,8,10.0
8,3,9,2022,16,8.0
7,3,1,2023,12,12.0


## rolling with real dataset

In [23]:
df_test = df_month_kg_ad.copy()


In [24]:
# STOK_NO, YIL ve AY sütunlarına göre sıralama
df_test = df_test.sort_values(by=['STOK_NO', 'YIL', 'AY'])

# STOK_NO'ya göre gruplama
grouped = df_test.groupby('STOK_NO')

# Her bir grup üzerinde OB1 için 3 aylık pencere üzerinde rolling işlemi yapma
df_test['OB1_3_Aylık_Rolling'] = grouped['OB1_AYLIK_ORTALAMA'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True).shift(1).fillna(0)
df_test['OB1_6_Aylık_Rolling'] = grouped['OB1_AYLIK_ORTALAMA'].rolling(window=6, min_periods=1).mean().reset_index(level=0, drop=True).shift(1).fillna(0)

# Her bir grup üzerinde OB2 için 3 aylık pencere üzerinde rolling işlemi yapma
# df_test['OB2_3_Aylık_Rolling'] = grouped['OB2_AYLIK_ORTALAMA'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True).shift(1).fillna(0)
# df_test['OB2_6_Aylık_Rolling'] = grouped['OB2_AYLIK_ORTALAMA'].rolling(window=6, min_periods=1).mean().reset_index(level=0, drop=True).shift(1).fillna(0)


In [25]:
df_test.head()

,STOK_NO,AY,YIL,OB1_AYLIK_ORTALAMA,OB1_3_Aylık_Rolling,OB1_6_Aylık_Rolling
5143,1384,3,2016,224.40,0.000000,0.0000
6807,1384,7,2016,172.20,224.400000,224.4000
8391,1384,11,2016,211.05,198.300000,198.3000
9159,1384,1,2017,464.80,202.550000,202.5500
11349,1384,7,2017,362.40,282.683333,268.1125


In [26]:
df_test.to_excel('C:\\Users\\osungar\\Desktop\projects\\sales_forecasting\\sales_project\\data\\df_stok_only.xlsx', index=False)


# TRAINING A LGB MODEL

In [27]:
categorical_columns=['STOK_NO', 'AY']
for col in categorical_columns:
    df_test[col] = df_test[col].astype('category')

In [28]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35367 entries, 5143 to 35366
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   STOK_NO              35367 non-null  category
 1   AY                   35367 non-null  category
 2   YIL                  35367 non-null  int64   
 3   OB1_AYLIK_ORTALAMA   35367 non-null  float64 
 4   OB1_3_Aylık_Rolling  35367 non-null  float64 
 5   OB1_6_Aylık_Rolling  35367 non-null  float64 
dtypes: category(2), float64(3), int64(1)
memory usage: 1.5 MB


In [29]:
X = df_test.drop(columns=['OB1_AYLIK_ORTALAMA'])

y = df_test['OB1_AYLIK_ORTALAMA']

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [52]:
X_test

,STOK_NO,AY,YIL,OB1_3_Aylık_Rolling,OB1_6_Aylık_Rolling
10994,747057,5,2017,996.700000,996.700000
14039,744520,2,2018,767.668889,576.817778
15624,2596,7,2018,31.233333,44.187500
16352,1739,9,2018,824.982011,873.637024
28749,2465,1,2022,318.511706,290.656687
...,...,...,...,...,...
16956,3017,10,2018,26.750000,30.200000
1682,744520,7,2015,515.450000,515.450000
34395,3201,7,2023,145.036111,144.374246
15098,4209,5,2018,48.416667,46.516667


In [53]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train,categorical_feature=categorical_columns)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [54]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'random_state':42
}

In [55]:
# Light Gradient Boosting Regressor
lgb_trained =lgb.train(params,
                lgb_train)


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000352 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1804
[LightGBM] [Info] Number of data points in the train set: 28293, number of used features: 5
[LightGBM] [Info] Start training from score 422.297080


In [56]:
y_pred =lgb_trained.predict(X_test)


In [57]:
X_test

,STOK_NO,AY,YIL,OB1_3_Aylık_Rolling,OB1_6_Aylık_Rolling
10994,747057,5,2017,996.700000,996.700000
14039,744520,2,2018,767.668889,576.817778
15624,2596,7,2018,31.233333,44.187500
16352,1739,9,2018,824.982011,873.637024
28749,2465,1,2022,318.511706,290.656687
...,...,...,...,...,...
16956,3017,10,2018,26.750000,30.200000
1682,744520,7,2015,515.450000,515.450000
34395,3201,7,2023,145.036111,144.374246
15098,4209,5,2018,48.416667,46.516667


In [48]:
print('MSE score on train data:')
print(mean_squared_error(y_test,y_pred))

MSE score on train data:
291842.5446977774


C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [112]:
153265
299819
150485

150485

In [36]:
merged_array = np.stack((y_test, y_pred), axis=1)


In [37]:
print("         y_test      ,      y_pred")
merged_array[:50]

         y_test      ,      y_pred


array([[ 318.4       ,  353.30372949],
       [ 359.41111111,  270.43736663],
       [ 427.4       , 1557.84156859],
       [ 392.5       ,  368.20297055],
       [ 215.58571429,  149.06216306],
       [  64.2       , 2329.87172319],
       [ 114.625     ,  261.3221108 ],
       [  33.75      ,  201.92118881],
       [ 719.1       ,  314.88065941],
       [  49.55      ,  177.18418651],
       [  88.025     ,  138.94386347],
       [  74.65      ,   87.53591822],
       [  40.95      ,   93.47472943],
       [  33.1       ,   67.20031927],
       [  58.35      ,  491.65208904],
       [ 461.05      ,  805.58281176],
       [ 188.05      ,  134.51375141],
       [ 331.98      ,  273.98673251],
       [ 783.875     ,  884.59440231],
       [  52.65      ,  266.35715181],
       [ 153.5875    ,  340.04382227],
       [1093.7       , 1266.15106923],
       [ 207.275     ,  191.86282986],
       [ 140.55      ,  125.02269268],
       [  65.9       ,  138.13017104],
       [ 243.9       ,  1

# KFold Cross Validation

In [39]:
categorical_columns=['STOK_NO', 'AY']
for col in categorical_columns:
    df_test[col] = df_test[col].astype('category')
    
X = df_test.drop(columns=['OB1_AYLIK_ORTALAMA'])

y = df_test['OB1_AYLIK_ORTALAMA']

In [40]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'random_state':42
}

In [41]:
from sklearn.model_selection import KFold


In [42]:
from sklearn.model_selection import KFold

mse_values = []  # Her iterasyonda elde edilen MSE değerlerini saklamak için liste
kf = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # create dataset for lightgbm
    lgb_train = lgb.Dataset(X_train, y_train,categorical_feature=categorical_columns)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    # LightGBM Regressor modelini oluştur
    
    
    # Light Gradient Boosting Regressor
    lgb_trained =lgb.train(params,
                lgb_train)

    y_pred =lgb_trained.predict(X_test)
    
    # Performans ölçümü (Örneğin, ortalama karesel hata)
    mse = mean_squared_error(y_test, y_pred)
    print(mse)
    mse_values.append(mse)

# K-Fold Cross Validation sonrasında MSE değerlerinin ortalamasını hesapla
average_mse = sum(mse_values) / len(mse_values)
print(f'Average Mean Squared Error: {average_mse}')

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000165 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1854
[LightGBM] [Info] Number of data points in the train set: 31830, number of used features: 5
[LightGBM] [Info] Start training from score 417.486791
267962.8102711156
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000146 seconds.
You can s

C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension

261351.16127138486
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000176 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1849
[LightGBM] [Info] Number of data points in the train set: 31830, number of used features: 5
[LightGBM] [Info] Start training from score 418.335768
239949.36757974242
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.00012

C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension

314325.92155680555
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000436 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1841
[LightGBM] [Info] Number of data points in the train set: 31830, number of used features: 5
[LightGBM] [Info] Start training from score 415.744654
273191.5186134667
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000194 seconds.
You can set `force_row_wise=true` to remove the overhe

C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension

273002.092308915
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000156 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1843
[LightGBM] [Info] Number of data points in the train set: 31830, number of used features: 5
[LightGBM] [Info] Start training from score 416.643816
354427.2016286165
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000148 s

C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension

277446.98733379407
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000147 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1849
[LightGBM] [Info] Number of data points in the train set: 31831, number of used features: 5
[LightGBM] [Info] Start training from score 414.921499
302737.06133646524
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.00012

C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension

In [ ]:
151859 = Basic model
280087 = only_stok

# PREDICTING 11/2023

In [ ]:
query = """
SELECT DISTINCT CARI_NO,STOK_NO     
FROM df_test
"""

# Run the query
df_dump = sqldf(query, locals())
df_dump.head(10)

In [ ]:
df_dump["AY"] = 11
df_dump["YIL"] = 2023
df_dump["OB2_AYLIK_ORTALAMA"] = 1
df_dump["OB1_AYLIK_ORTALAMA"] = 1


In [ ]:
df_dump.head()

In [ ]:
merge1= df_month_kg_ad.copy()

In [ ]:
result_df = pd.concat([merge1, df_dump], axis=0, ignore_index=True)


In [ ]:
query = """
SELECT STOK_NO,AY,YIL,OB2_AYLIK_ORTALAMA,OB1_AYLIK_ORTALAMA     
FROM result_df
group by STOK_NO,YIL,AY
order by YIL,AY ASC
"""

# Run the query
result_df = sqldf(query, locals())
result_df.head(10)

In [ ]:
# ROLLING YAP